In [ ]:
import numpy as np

# define the activation functions

In [ ]:
def sigmoid(x):
    return (np.exp(x)-1)/(np.exp(x)+1)

In [ ]:
def hyperbolic_tangent(x):
    return (np.exp(2*x)-1)/(np.exp(2*x)+1)

In [ ]:
def logistic(x):
    return np.exp(x)/(np.exp(x)+1)

In [ ]:
def softplus(x):
    return np.log(np.exp(x)+1)

# define an one-hidden layer neural network with  r nodes

In [ ]:
# v is the output weights, which is an r-dimensional vector
# w is the input weights, which is an r*n matrix
# b is the input biases, which is an r-dimensional vector
# d is the output bias, which is a scalar

In [ ]:
# for testing, give the initial values of variables, set r = 3, and n = 2.
# v = np.array([[1,2,3]]).T
# w = np.np.array([[10,2],[2,13],[1,9]])
# x = np.array([[0.2,0.5]]).T
# b = np.array([[1,-5,0]]).T
# d = 5

In [ ]:
def neural_network(sigma,v,w,x,b,d):
    inner_vector = np.matmul(w,x)+b
    if sigma == sigmoid:
        return np.matmul(v.T,sigmoid(inner_vector))+d
    elif sigma == hyperbolic_tangent:
        return np.matmul(v.T,hyperbolic_tangent(inner_vector))+d
    elif sigma == logistic:
        return np.matmul(v.T,logistic(inner_vector))+d
    elif sigma == softplus:
        return np.matmul(v.T,softplus(inner_vector))+d

# define the matrix A

## First, define the partial derivatives of the neural network, i.e. Jacobian matrix

In [ ]:
def N_v (sigma,v,w,x,b,d):
    inner_vector = np.matmul(w,x)+b
    if sigma == sigmoid:
        return sigmoid(inner_vector).T
    elif sigma == hyperbolic_tangent:
        return hyperbolic_tangent(inner_vector).T
    elif sigma == logistic:
        return logistic(inner_vector).T
    elif sigma == softplus:
        return softplus(inner_vector).T

In [ ]:
def dsigmoid(x):
    return 2*np.exp(x)/(np.exp(x)+1)**2

In [ ]:
def dhyperbolic_tangent(x):
    return 4*np.exp(2*x)/(np.exp(2*x)+1)**2

In [ ]:
def dlogistic(x):
    return np.exp(x)/(np.exp(x)+1)**2

In [ ]:
def dsoftplus(x):
    return np.exp(x)/(np.exp(x)+1)

In [ ]:
def N_w(sigma,v,w,x,b,d,i):
    inner_vector = np.matmul(w,x)+b
    if sigma == sigmoid:
        return (v*dsigmoid(inner_vector)*x[i-1,0]).T
    elif sigma == hyperbolic_tangent:
        return (v*dhyperbolic_tangent(inner_vector)*x[i-1,0]).T
    elif sigma == logistic:
        return (v*dlogistic(inner_vector)*x[i-1,0]).T
    elif sigma == softplus:
        return (v*dsoftplus(inner_vector)*x[i-1,0]).T

In [ ]:
def N_b(sigma,v,w,x,b,d):
    inner_vector = np.matmul(w,x)+b
    if sigma == sigmoid:
        return (v*dsigmoid(inner_vector)).T
    elif sigma == hyperbolic_tangent:
        return (v*dhyperbolic_tangent(inner_vector)).T
    elif sigma == logistic:
        return (v*dlogistic(inner_vector)).T
    elif sigma == softplus:
        return (v*dsoftplus(inner_vector)).T

## Define matrix A, which is an "approximation" of matrix B=J^TJ, considering the practical meaning of the parameters, we need to consider the problem with triples {v,w,b}

In [ ]:
# use np.linalg.norm(N_ξ,1) to calculate the infinity norm of the row vector

In [ ]:
def N_w_sum(sigma,v,w,x,b,d):
    sum_w = 0
    for i in range(1,len(x)+1):
        sum_w += np.matmul(N_w(sigma,v,w,x,b,d,i).T,N_w(sigma,v,w,x,b,d,i))/np.linalg.norm(N_w(sigma,v,w,x,b,d,i),1)
    return sum_w

In [ ]:
def matrix_A(sigma,v,w,x,b,d):
    return np.matmul(N_b(sigma,v,w,x,b,d).T,N_b(sigma,v,w,x,b,d))/np.linalg.norm(N_b(sigma,v,w,x,b,d),1)+\
           np.matmul(N_v(sigma,v,w,x,b,d).T,N_v(sigma,v,w,x,b,d))/np.linalg.norm(N_v(sigma,v,w,x,b,d),1)+\
           N_w_sum(sigma,v,w,x,b,d)

# C/F-splitting

In [ ]:
# Reference for python: https://github.com/pyamg/pyamg/blob/main/pyamg/classical/split.py

In [ ]:
# Reference for Julia: https://github.com/JuliaLinearAlgebra/AlgebraicMultigrid.jl/blob/master/src/splitting.jl

In [ ]:
import pyamg

In [ ]:
from pyamg import classical
from pyamg.classical.split import RS
from pyamg.blackbox import make_csr

In [ ]:
#RS(matrix_A(sigma,v,w,x,b,d),second_pass=False)
# Here A must be a csr matrix, so we use make_csr to convert matrix A into a csr matrix

In [ ]:
A_csr=make_csr(matrix_A(sigmoid,v,w,x,b,d)) #here we choose the sigmoid function as the activation function 
RS(A_csr,second_pass=False)